In [9]:
import os
import subprocess
import pandas as pd
import numpy as np
from dataclasses import dataclass



In [21]:
@dataclass
class Command:
    vocab_file: str
    dict_file: str
    use_gpu: bool = False
    interactive: bool = False

def form_command(c: Command): 
    cmd = [f'./solver -d {c.dict_file} -v {c.vocab_file}']
    if c.use_gpu:
        cmd.append('-g')
    if c.interactive:
        cmd.append('-i')
    return ' '.join(cmd).split(' ')

def run_command(c: Command):
    cmd = form_command(c)
    output = subprocess.check_output(cmd).decode()
    print(output)

In [24]:
c = Command(vocab_file='./basic_dictionary/vocab.txt',
            dict_file = './basic_dictionary/potential_words.txt',
            use_gpu=True)

run_command(c)

Target Word: rainy

Occupancy: 0.476888
Expected Information: 5.9392
Solver Iteration Time: 1942220958
Solver guessed: slate
Guess: Colors: 
0 0 2 0 0 

Actual Information: 4.08933
Occupancy: 0.0280219
Expected Information: 5.45817
Solver Iteration Time: 296660761
Solver guessed: manor
Guess: Colors: 
0 1 2 0 2 

Actual Information: 6.08746
Occupancy: 0.000434028
Expected Information: 1
Solver Iteration Time: 1786564
Solver guessed: cairn
Guess: Colors: 
0 1 1 2 2 

Actual Information: 1
Occupancy: 0.000217014
Expected Information: 0
Solver Iteration Time: 206563
Solver guessed: rainy
Guess: Colors: 
1 1 1 1 1 

Solved in 4
End to End Time: 2253423585

